## Canvas Data:

CSV with tuples <idx, time, user, color(hex), x_coordinate, y_coordinate>

Previous code data template <time, user, x_coordinate, y_coordinate, color>

Data stored in 

../data_2022/tile_placement.csv

../data_2023/tile_placement.csv

In [1]:
#import basic libraries

import csv
import sys
import os
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import axes3d
#%matplotlib tk
import operator
sys.path.append("../Python_code") # go to parent dir
from reddit import *
from analytics_combined import *
import pandas as pd

# Process 2022 data to match format

In [22]:
# Process the data so that it matches the original file format
# Cuts the index
!cut -f2,3,4,5,6 -d ',' ../data_2022/tile_placement.csv > ../data_2022/tile_placement_new.csv

In [26]:
# Swap color and coord column
!awk -F, 'BEGIN{FS=OFS=","} {print $1, $2, $4, $5, $3}' ../data_2022/tile_placement_new.csv > ../data_2022/tile_placement_swap.csv

In [27]:
# Removes the quotation between coordinates
!awk '{gsub(/"/, "")};4' ../data_2022/tile_placement_swap.csv > ../data_2022/tile_placement_noquote.csv

In [28]:
# the original tile_placements.csv contains duplicate updates (same [ts, user, x, y, color]) 
# One needs to run the following command to only keep the unique updates
!awk '!a[$0]++' ../data_2022/tile_placement_noquote.csv > ../data_2022/tile_placements_no_duplicates.csv

In [13]:
# Delete the UTC at the end of each timestamp
!awk '{gsub("UTC", "")$1;print}' ../data_2022/tile_placements_no_duplicates.csv > ../data_2022/tile_placements_no_UTC.csv

In [20]:
# Convert UTC time to unix timestamp

!awk -F, 'BEGIN{FS=OFS=","} {$1 = mktime(gensub("[-:]", " ","g",$1))} 1' ../data_2022/tile_placements_no_UTC.csv > ../data_2022/tile_placements_time.csv

In [22]:
# Remove the '#' in front of the hex colors
!awk '{gsub("#", "")$5;print}' ../data_2022/tile_placements_time.csv > ../data_2022/tile_placements_color.csv

In [ ]:
# Takes the tile_placements and atlas data and creates a csv where each line is a tuple of the following elements:
# time, user, x, y, color, project_id, pixel (binary), pixel_color (binary)
# pixel denotes whether it is in the final project
# pixel_color denotes whether the update has the same color as the final one or not
locations = store_locations_22("../data_2022/atlas/atlas.json")
print("locations length:", len(locations))
add_atlas_data_to_tile_placements(locations, "../data_2022/tile_placements_color.csv", "../data_2022/tile_placements_proj.csv")

locations length: 10866


In [3]:
# One needs to sort the data according to the time
# Please keep the header as it is; the following command will help to sort.
# We will use the sorted_tile_placements.csv

!awk 'NR == 1; NR > 1 {print $0 | "sort -t, -nk1,1"}' ../data_2022/tile_placements_proj.csv > ../data_2022/sorted_tile_placements_proj.csv

sort: write failed: /tmp/sortfJDg1x: No space left on device
awk: cmd. line:1: (FILENAME=../data_2022/tile_placements_proj.csv FNR=521296105) fatal: print to "sort -t, -nk1,1" failed (Broken pipe)


# Process 2023 data to match format

In [30]:
# Process the data so that it matches the original file format
# Cuts the index
!cut -f2,3,4,5,6,7 -d ',' ../data_2023/tile_placement.csv > ../data_2023/tile_placement_new.csv

In [31]:
# Remove quotation and make sure the format is consistent in the file
!awk '{gsub(/, R: [0-9]*}/,""); gsub(/{X: /, "");gsub(/Y: /, ""); gsub(/"/, "")} 4;' ../data_2023/tile_placement_new.csv > ../data_2023/tile_placement_noquote.csv

In [32]:
# the original tile_placements.csv contains duplicate updates (same [ts, user, x, y, color]) 
# One needs to run the following command to only keep the unique updates
!awk '!a[$0]++' ../data_2023/tile_placement_noquote.csv > ../data_2023/tile_placements_no_duplicates.csv

In [33]:
# Delete the UTC at the end of each timestamp

!awk '{gsub("UTC", "")$1;print}' ../data_2023/tile_placements_no_duplicates.csv > ../data_2023/tile_placements_no_UTC.csv

# Convert UTC time to unix timestamp

!awk -F, 'BEGIN{FS=OFS=","} {$1 = mktime(gensub("[-:]", " ","g",$1))} 1' ../data_2023/tile_placements_no_UTC.csv > ../data_2023/tile_placements_time.csv

In [34]:
# Remove the '#' in front of the hex colors
!awk '{gsub("#", "")$5;print}' ../data_2023/tile_placements_time.csv > ../data_2023/tile_placements_color.csv

In [ ]:
# Takes the tile_placements and atlas data and creates a csv where each line is a tuple of the following elements:
# time, user, x, y, color, project_id, pixel (binary), pixel_color (binary)
# pixel denotes whether it is in the final project
# pixel_color denotes whether the update has the same color as the final one or not
locations = store_locations_22("../data_2023/atlas/atlas.json")
print("locations length:", len(locations))
add_atlas_data_to_tile_placements(locations, "../data_2023/tile_placements_color.csv", "../data_2023/tile_placements_proj.csv")

In [28]:
# Process the data so that it matches the original file format
# Cuts the index
!cut -f2,3,4,5,6,7 -d ',' test.csv > test_out.csv

In [ ]:
with open("../data_2023/atlas/atlas.json",'r') as file:
    next(file, None)
    reader = csv.reader(file)

    with open("../data_2023/sorted_tile_placements_coord.csv", 'w') as file_out:
      writer = csv.writer(file_out, delimiter = ",")
      writer.writerow(["ts", "user" ,"x_coordinate" ,"y_coordinate" ,"color"])

      for r in reader:
        time = int(r[0])
        user = r[1]
        x = int(r[2]) + 1500
        y = int(r[3]) + 1000
        color = r[4]
        writer.writerow([time, user, x, y, color])